In [28]:
import os
import pandas as pd
from google.cloud import storage
from google.oauth2.service_account import Credentials

In [29]:
scopes = ["https://www.googleapis.com/auth/cloud-platform"]

SERVICE_ACCOUNT_FILE = "/Data2/Arun-UAV/NLP/self-halu-detection/vertix_ai.json"

credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=scopes)

In [30]:
client = storage.Client(credentials=credentials)
bucket_name = "train_data_vision"
bucket = client.bucket(bucket_name)

In [35]:
from tqdm import tqdm
def upload_images_to_gcs(folder_path: str, bucket_name: str, prefix: str = "") -> pd.DataFrame:
    """
    Uploads all images from a local folder to a specified GCS bucket.

    Args:
        folder_path (str): Local folder containing images.
        bucket_name (str): Target GCS bucket name.
        prefix (str): Optional folder prefix in the bucket (e.g. "images/").

    Returns:
        pd.DataFrame: DataFrame with columns [local_path, gcs_uri].
    """
    # Initialize client (ensure GOOGLE_APPLICATION_CREDENTIALS is set)


    # Collect image files
    image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp"}
    files = [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if os.path.splitext(f)[1].lower() in image_extensions
    ]

    upload_data = []

    for file_path in tqdm(files):
        blob_name = os.path.join(prefix, os.path.basename(file_path))
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(file_path)
        gcs_uri = f"gs://{bucket_name}/{blob_name}"
        upload_data.append((file_path, gcs_uri))

    df = pd.DataFrame(upload_data, columns=["local_path", "gcs_uri"])
    return df


In [36]:
df = upload_images_to_gcs(folder_path="/Data2/Arun-UAV/NLP/vision_halu/train_datasets/poc_5000_coco_images", bucket_name=bucket_name)

  0%|          | 0/4906 [00:00<?, ?it/s]

 10%|█         | 514/4906 [09:23<1:20:10,  1.10s/it]


KeyboardInterrupt: 

In [32]:
folder_path="/Data2/Arun-UAV/NLP/vision_halu/train_datasets/poc_5000_coco_images"
prefix = ""
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp"}
files = [
    os.path.join(folder_path, f)
    for f in os.listdir(folder_path)
    if os.path.splitext(f)[1].lower() in image_extensions
]

upload_data = []

for file_path in files:
    blob_name = os.path.join(prefix, os.path.basename(file_path))
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(file_path)
    gcs_uri = f"gs://{bucket_name}/{blob_name}"
    upload_data.append((file_path, gcs_uri))
    break
        

In [38]:
df = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/train_datasets/coco_5000_gcp_upload_urs.csv")

In [43]:
df["gcs_uri"].iloc[0]

'gs://train_data_vision/poc_coco_5000/COCO_train2014_000000217322.jpg'